# MDMS Meter Consumption Statistical Filter

In [1]:
from sklearn import preprocessing as pp
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import chart_studio as cs
init_notebook_mode(connected=True)

import cufflinks as cf
print(cf.__version__)
cf.go_offline()

cs.tools.set_credentials_file(username='arnoc', api_key='rqPrzFeAXuVfUIRKnwLC')
cs.tools.set_config_file(world_readable=True)

%matplotlib inline

0.16


<h3>INITIALIZE MDMS DB METHODS<h3/>

In [2]:
# Symbolic link from /usr/lib64/libnsl.so.2.0.0 to /usr/lib64/libnsl.so.1 required

from __future__ import print_function

#import os
#os.chdir("/usr/lib/oracle/19.3/client64/lib")

import traceback
import cx_Oracle
import numpy as np
import pandas as pd
import seaborn as sns
import time
import sys

def connectToOracleDB():
    global connection
    global cursor
    global username
    global password
    global host
    global service
    
    connection = 0
    cursor = 0
    
    host     = "dboramdms.spsi.co.za"
    username = "icemdmsudev"
    password = "V3ry$3Cr!t"
    service  = "mdms"

    
    try:
        
        connection = cx_Oracle.connect(username, password, host+"/"+service)
        
        cursor = connection.cursor()
        
        print("\nOracle DB connection initialized\n")
        
    except Exception as err:
        exc_type, exc_value, exc_tb = sys.exc_info()
        traceback.print_exception(exc_type, exc_value, exc_tb)
        exit()
           
def executeSQLquery( sql ):
    global cursor
    
    print("{}".format(sql))
    
    cursor.execute( sql )
    
    return cursor.fetchall()

def getMeterReadings( meterType, meterID, fromDt, toDt ):

    #meterID = "36a878da44b6434fbe12173931dceaf1"
    #fromDt = "2019-10-01 00:00:00" 
    #toDt = "2019-10-31 23:59:59"
    
    meterTypeCol = ""
    
    if( meterType == 'ELSTER'):
        meterTypeCol = "ELS_METER_ID"
    elif( meterType == "KAMSTRUP" ):
        meterTypeCol = "KAM_METER_ID"
    elif( meterType == "ECHELON" ):
        meterTypeCol = "NES_METER_ID"
    elif( meterType == "GENERIC" ):
        meterTypeCol = "GENERIC_METER_ID"
    
    results = executeSQLquery( """SELECT ENTRY_TIME, TOTAL_KWHP, TOTAL_KWHN, TOTAL_KVARP, TOTAL_KVARN FROM METER_READING
                                    WHERE {} = '{}'
                                    AND ENTRY_TIME BETWEEN TO_DATE('{}','YYYY-MM-DD HH24:MI:SS') AND TO_DATE('{}','YYYY-MM-DD HH24:MI:SS') 
                                    ORDER BY ENTRY_TIME ASC"""
                              .format( meterTypeCol, meterID, fromDt, toDt ) )
    
    #results = executeSQLquery( """SELECT * FROM ( SELECT METER_READING_ID, ENTRY_TIME, TOTAL_KWHP, TOTAL_KWHN, TOTAL_KVARP, TOTAL_KVARN FROM METER_READING
    #                               WHERE NES_METER_ID = '{}'
    #                              AND ( ENTRY_TIME BETWEEN TO_DATE('{}','YYYY-MM-DD HH24:MI:SS') AND
    #                                TO_DATE('{}','YYYY-MM-DD HH24:MI:SS')) ORDER BY ENTRY_TIME ASC )
    #                                WHERE ROWNUM <= {:d}"""
    #                          .format( meterID, fromDt, toDt, days*48 ) )
    
    return results

print("MDMS Functions Initialized")

MDMS Functions Initialized


In [3]:
connectToOracleDB()


Oracle DB connection initialized



In [15]:
#### KAMSTRUP METERS
#mid = '5a17e029-cc82-40ab-80d9-b0aea5ccb05d' #30829071
#mid = '4a6baa3d-5463-409e-9789-4a323dc0b6b4' #30829119
#mid = 'a5c318f6-0541-414b-a43d-53e599168be4' #30829087
#mid = 'd4c03484-55d9-4b0e-a9e6-3ec9d7e9716d' #19675054

#### ELSTER METERS
#mid = '0cc19758-faee-400f-b221-d608ff952e56' #3514281191616
#mid = '7bc626cc-227f-4fa5-85e1-54ed21c9dec0' #3514885019338
#mid = 'e6f871d6-c1bb-4e29-b171-d23fd6ecbfb4' #3514111291545
#mid = '2943b027-74b8-4b9f-bb75-baf92a9b45d9' #3514885224631
#mid = '69f182b3-86db-43c6-bfb1-d874f746bf4b' #3514081026427
mid = 'e3b81a0d-5434-41f2-aabc-f3fa0eda30a3' #3514885012630

#### ECHELON METERS


#### GENERIC METERS

meterReadings = getMeterReadings( meterType="ELSTER", meterID=mid, fromDt='2019-11-01 00:00:00', toDt='2019-11-30 23:59:59')
meterReadings = pd.DataFrame(data=meterReadings, columns=['ENTRY_TIME','TOTAL_KWHP','TOTAL_KWHN','TOTAL_KVARP','TOTAL_KVARN'])
meterReadings.head(10)

SELECT ENTRY_TIME, TOTAL_KWHP, TOTAL_KWHN, TOTAL_KVARP, TOTAL_KVARN FROM METER_READING
                                    WHERE ELS_METER_ID = 'e3b81a0d-5434-41f2-aabc-f3fa0eda30a3'
                                    AND ENTRY_TIME BETWEEN TO_DATE('2019-11-01 00:00:00','YYYY-MM-DD HH24:MI:SS') AND TO_DATE('2019-11-30 23:59:59','YYYY-MM-DD HH24:MI:SS') 
                                    ORDER BY ENTRY_TIME ASC


,ENTRY_TIME,TOTAL_KWHP,TOTAL_KWHN,TOTAL_KVARP,TOTAL_KVARN
0,2019-11-01 00:00:00,2.065628e+07,0.0,5289062.945,0.0
1,2019-11-01 00:30:00,2.065638e+07,0.0,5289066.792,0.0
2,2019-11-01 01:00:00,2.065649e+07,0.0,5289071.647,0.0
3,2019-11-01 01:30:00,2.065661e+07,0.0,5289078.427,0.0
4,2019-11-01 02:00:00,2.065674e+07,0.0,5289088.325,0.0
5,2019-11-01 02:30:00,2.065685e+07,0.0,5289094.791,0.0
6,2019-11-01 03:00:00,2.065696e+07,0.0,5289102.310,0.0
7,2019-11-01 03:30:00,2.065707e+07,0.0,5289107.007,0.0
8,2019-11-01 04:00:00,2.065721e+07,0.0,5289111.417,0.0
9,2019-11-01 04:30:00,2.065746e+07,0.0,5289115.208,0.0


In [16]:
meterCons = meterReadings.diff()

mergedCons = pd.merge( meterCons, meterReadings['ENTRY_TIME'], left_index=True, right_index=True )

mergedCons.drop('ENTRY_TIME_x', axis=1, inplace=True)

mergedCons = mergedCons[1:]

mergedCons = mergedCons.rename(columns={'ENTRY_TIME_y':'TIMESTAMPS'})

mergedCons = mergedCons.set_index('TIMESTAMPS')

#mergedCons = mergedCons.loc['2019-12-01 00:00:00':'2019-12-31 23:59:59']

mergedCons['KVA'] = np.sqrt( np.square( 2*( mergedCons['TOTAL_KWHP'] - mergedCons['TOTAL_KWHN'] ) ) + np.square( 2*( mergedCons['TOTAL_KVARP'] - mergedCons['TOTAL_KVARN'] ) ) )

mergedCons = mergedCons.reset_index()

mergedCons.drop_duplicates(subset ="TIMESTAMPS", keep = 'first', inplace = True) 

mergedCons = mergedCons.set_index('TIMESTAMPS')

meterCons = mergedCons.copy()

meterCons.head()

,TOTAL_KWHP,TOTAL_KWHN,TOTAL_KVARP,TOTAL_KVARN,KVA
TIMESTAMPS,,,,,
2019-11-01 00:30:00,104.798,0.0,3.847,0.0,209.737171
2019-11-01 01:00:00,107.371,0.0,4.855,0.0,214.961417
2019-11-01 01:30:00,120.066,0.0,6.780,0.0,240.514555
2019-11-01 02:00:00,123.432,0.0,9.898,0.0,247.656448
2019-11-01 02:30:00,116.646,0.0,6.466,0.0,233.650153


In [17]:
min_max_scaler = pp.MinMaxScaler()
kva_minmax = min_max_scaler.fit_transform(meterCons)
meterConsMinMax = pd.DataFrame(kva_minmax,columns=meterCons.columns) 
meterConsMinMax.head()

,TOTAL_KWHP,TOTAL_KWHN,TOTAL_KVARP,TOTAL_KVARN,KVA
0,0.000000,0.0,0.014977,0.0,0.000000
1,0.002765,0.0,0.022287,0.0,0.002802
2,0.016406,0.0,0.036249,0.0,0.016507
3,0.020023,0.0,0.058862,0.0,0.020338
4,0.012731,0.0,0.033971,0.0,0.012826


In [18]:
ufResults = meterCons.copy()

ufMaxKva = ufResults['KVA'].max()

print("UNFILTERED MAX KVA: {}".format(ufMaxKva))

q3filter = 0.85

fResults = ufResults[ ufResults['KVA'] <= ufResults['KVA'].quantile(q3filter) ]

fMaxKva = fResults['KVA'].max()

print("FILTERED MAX KVA: {}" \
      .format(fMaxKva))

print("TOTAL KVA MEAN: {}" \
      .format(ufResults['KVA'].mean()))

print("TOTAL KVA STANDARD DEVIATION: {}" \
      .format(ufResults['KVA'].std()))

print("Total KVA Outlier Value (mean + 3 * standard deviation): {}" \
      .format(ufResults['KVA'].mean() + 3 * ufResults['KVA'].std()))

print("Total KVA Outlier Value (Q3 @ 85%): {}" \
      .format(ufResults['KVA'].quantile(q3filter)))


UNFILTERED MAX KVA: 2074.2131853222572
FILTERED MAX KVA: 483.3048786274607
TOTAL KVA MEAN: 434.6294308129663
TOTAL KVA STANDARD DEVIATION: 250.84089118022337
Total KVA Outlier Value (mean + 3 * standard deviation): 1187.1521043536363
Total KVA Outlier Value (Q3 @ 85%): 483.3297965761078


In [19]:
otResults = ufResults[ ufResults['KVA'] > ufResults['KVA'].quantile(q3filter) ]
otResults

,TOTAL_KWHP,TOTAL_KWHN,TOTAL_KVARP,TOTAL_KVARN,KVA
TIMESTAMPS,,,,,
2019-11-01 04:30:00,244.346,0.0,3.791,0.0,488.750813
2019-11-01 07:00:00,383.516,0.0,10.736,0.0,767.332481
2019-11-01 09:00:00,245.238,0.0,24.133,0.0,492.845121
2019-11-01 11:00:00,653.794,0.0,72.185,0.0,1315.533760
2019-11-01 14:00:00,242.832,0.0,27.399,0.0,488.745682
2019-11-01 23:30:00,476.102,0.0,50.374,0.0,957.518990
2019-11-02 03:30:00,651.360,0.0,48.129,0.0,1306.271412
2019-11-02 08:00:00,242.547,0.0,25.502,0.0,487.767974
2019-11-02 09:00:00,462.549,0.0,48.803,0.0,930.232896


In [20]:
ufResults['KVA'].iplot(kind='bar', yTitle='Total KVA', title='Unfiltered Apparent Power Profile (KVA)')
print("UNFILTERED TOTAL KVA MAX: {:.3f}".format( ufResults['KVA'].max() ))

UNFILTERED TOTAL KVA MAX: 2074.213


In [21]:
ufResults['KVA'].iplot(kind='histogram', yTitle='Total KVA', title='Unfiltered Apparent Power Histogram (KVA)')

In [22]:
fResults['KVA'].iplot(kind='bar', yTitle='Total KVA', title='Filtered Apparent Power Profile (KVA)')
print("FILTERED TOTAL KVA MAX: {:.3f}".format( fResults['KVA'].max() ))

FILTERED TOTAL KVA MAX: 483.305


In [23]:
fResults['KVA'].iplot(kind='histogram', yTitle='Total KVA', title='Filtered Apparent Power Histogram (KVA)')

In [13]:
#fResults['WEEKNAMES'] = fResults.index.weekday_name
#fResults['YEAR']      =  fResults.index.year
#fResults['MONTH']     =  fResults.index.month
#fResults['DAY_OF_YEAR']     =  fResults.index.dayofyear

#fResults.loc['2019-12'].head()

In [14]:
#plt.plot(energyCons['ENTRY_TIME'],energyCons['TOTAL_KWHP'],label="KWHP")

#sns.set(rc={'figure.figsize':(20, 7)})
#sns.barplot(x="TIMESTAMPS", y="TOTAL_KWHP", data=results)

#daily_sums = fResults['TOTAL_KWHP'].resample('D').sum()
#daily_sums.head(20)

#ax = daily_sums.plot.bar(figsize=(23,5))
#ax.set_ylabel('Total Active kWh')
#ax.set_title('Energy Consumption Load Profile')
#plt.xticks(rotation=90)